London SCOOT (Split Cycle Offset Optimisation Technique) is a traffic signal control system designed to optimize urban road traffic flow by dynamically adjusting the timing of traffic signals. This technology was originally developed in the UK and has been widely implemented in cities like London. 

Here's a detailed introduction to this technology:

### How It Works

SCOOT uses ground sensors or cameras to collect real-time traffic data, such as the number of vehicles, their direction, and speed. This data is then transmitted to a central control system, where it is used to calculate the optimal signal light change scheme to reduce traffic congestion and improve road usage efficiency.

### Key Features

- Dynamic Optimization: 

    - The SCOOT system can dynamically adjust the phases and cycles of traffic lights based on real-time traffic conditions. This means that the operation of traffic lights can be adjusted in response to changes in traffic demand, such as during rush hours, holidays, or special events.

- Reduced Waiting Time: 

    - By optimizing the operation of traffic lights, SCOOT helps to reduce vehicle waiting times and queue lengths, thereby increasing the capacity of roads.

- Increased Traffic Efficiency: 

    - SCOOT aims to smooth traffic flow by reducing the number of stops and starts, thereby lowering fuel consumption and emissions from vehicles.

- Adaptability: 

    - The SCOOT system can adapt to various traffic conditions, including city streets, intersections, and multi-lane roads, making it a flexible tool for traffic management.

- Real-time Data Feedback: 

    - SCOOT provides real-time data analysis and feedback, helping traffic managers to monitor conditions and adjust strategies in response to emergencies.

### Case Studies

In London, the implementation of SCOOT technology has significantly improved urban traffic mobility. It has been deployed at numerous intersections and key roads, effectively alleviating traffic congestion, enhancing traffic efficiency, and positively impacting environmental protection.

### Summary

In summary, London SCOOT is an advanced traffic signal control system that significantly enhances the efficiency and fluidity of road networks through real-time data analysis and dynamic signal adjustments. Its successful implementation has not only improved traffic conditions in London but also serves as an important reference for traffic management in other cities.

### Data Preprocessing

We will do some preliminary data processing on the congestion scoot data. This data is downloaded from https://roads.data.tfl.gov.uk/. This data contains all the traffic congestion data from December 17, 2017 to March 16, 2018. We will do some preliminary processing on this data.


我们对 congestion scoot 这个数据进行初步的预处理。这个数据下载来自 https://roads.data.tfl.gov.uk/ 。这个数据包含了从2017年12月17日到2018年3月16日所有的交通拥堵数据。我们将这个数据进行初步的处理。

In [1]:
# import libraries

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
import contextily as ctx

import os
import glob

First, we will process the metadata. The metadata is a csv file that contains the observation point id and their coordinate information (represented in the British National Grid coordinate system).

首先是对metadata的处理，metadata是一个csv文件，里面包含了所有记录道路拥堵数据的观测点id以及他们的坐标信息（基于British National Grid坐标系统进行表示）。

In [2]:
file_path = '../Data/CongestionScoot/Metadata.csv'

# read the data
# 读取 CSV 文件
df = pd.read_csv(file_path)

df.sample(5)

,ID,Easting,Northing
1115,05-311,533600,182036
2871,19-004,541044,168196
3421,25-088,519130,178200
704,03-063,529347,186671
3867,30-010,525053,186441


In [3]:
# make sure the Easting and Northing columns are in the correct data type
# 确保 Easting 和 Northing 列的数据类型是正确的
df['Easting'] = pd.to_numeric(df['Easting'], errors='coerce')
df['Northing'] = pd.to_numeric(df['Northing'], errors='coerce')

In [5]:
# create a geometry column
# 创建点几何列
geometry = [Point(xy) for xy in zip(df['Easting'], df['Northing'])]

# create GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Easting'], df['Northing']))


# set(CRS) as (EPSG:27700)，and transfer into WGS84 (EPSG:4326)
gdf.crs = 'epsg:27700'
gdf = gdf.to_crs(epsg=4326)

# create folium map object
m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], # set the center of the map
    zoom_start=10 # initial zoom level
)

# plot every point in the GeoDataFrame
for _, row in gdf.iterrows():
    # the icons could be customized futurely
    icon = folium.Icon(
        icon='glyphicon glyphicon-map-marker', 
        prefix='glyphicon',  
        color='red',  
    )
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=icon,
        popup=str(row['ID'])  # assusming there is an 'ID' column as identifier, and make sure to convert it to string
    ).add_to(m)


# add the layer control to the map
folium.LayerControl().add_to(m)



# save the map to an HTML file
m.save('map.html')

首先需要对三个文件夹中的同名文件进行合并，然后对合并后的文件进行处理。

In [6]:
# 文件夹路径
folders = ['../Data/CongestionScoot/CSV-171217-180115', '../Data/CongestionScoot/CSV2-180116-180214', '../Data/CongestionScoot/CSV3-180215-180316']
new_folder = '../Data/CongestionScoot/MergedCSVs'

# 确保输出文件夹存在
if not os.path.exists(new_folder):
    os.makedirs(new_folder)


In [25]:
# 定义metadata中所有观测点的ID列表
all_ids = df['ID'].unique()

In [24]:
# 找到已有数据源中的所有观测点ID集合
def get_filenames_without_extension(directory):
    filenames = []  # 创建一个空列表，用于存储不带扩展名的文件名
    for filename in os.listdir(directory):
        # 使用 os.path.splitext 来分离文件名和扩展名
        base_name, _ = os.path.splitext(filename)
        filenames.append(base_name)  # 将不带扩展名的文件名添加到集合中
    return filenames

directory_path = '../Data/CongestionScoot/CSV-171217-180115/'
filenames = get_filenames_without_extension(directory_path)

In [26]:
setA = set(filenames)
setB = set(all_ids)
unique_ids = setA ^ setB
unique_ids

{'01-018',
 '01-470',
 '01-850',
 '01-857',
 '01-858',
 '01-859',
 '01-861',
 '01-863',
 '02-025',
 '02-079',
 '03-850',
 '04-102',
 '04-194',
 '05-852',
 '09-857',
 '12-173',
 '13-077',
 '13-172',
 '18-088',
 '23-850',
 '28-262',
 '32-019'}

In [34]:
# 因为csv文件前几行是无效的，所以需要跳过前几行
ObservationPoint = pd.read_csv('../Data/CongestionScoot/CSV-171217-180115/00-005.csv', skiprows=4)
ObservationPoint.sample(4)

,DateTime,Date,Time,SatMean,SatBand,FlowMean
2774,43114.885417,14-Jan-2018,21:15,65.25,0-79%,3998
1767,43104.395833,4-Jan-2018,09:30,100.00,>= 100%,5766
2644,43113.531250,13-Jan-2018,12:45,100.00,>= 100%,5882
1787,43104.604167,4-Jan-2018,14:30,60.50,0-79%,3480


In [27]:

# 对于每个共有的文件
for id in filenames:
    
    # 读取并合并文件
    dfs = []
    for folder in folders:
        file_path = f"{folder}/{id}.csv"
        # 检查文件是否存在
        if os.path.exists(file_path):
            # 忽略每个文件的前4行,因为这些行包含了不必要的元数据
            df = pd.read_csv(file_path, skiprows=4)
            dfs.append(df)
        else:
            print(f"File {file_path} not found, skipping.")
    
    # 保存到新文件夹
    if dfs:
        merged_df = pd.concat(dfs, ignore_index=True)
        # 保存到新文件夹
        merged_df.to_csv(f"{new_folder}/{id}.csv", index=False)

print("Merging completed.")

File ../Data/CongestionScoot/CSV3-180215-180316/32-194.csv not found, skipping.
Merging completed.


Since every obervation's recording is in a separate csv file, we will convert the metadata from long table to wide table. This will make it easier for us to process the time series data of each observation point.

因为每个记录点的congestion信息都是按照时间进行记录的，而每个记录点是一个单独的csv文件。如果需要对某个观测点的时间序列数据进行处理，我们需要对原csv文件进行某些预处理，将long table转换为wide table。这样我们就可以对某个观测点的时间序列数据进行处理。


In [50]:
ObservationPoint_pivot = ObservationPoint.pivot_table(index='Date', columns='Time', values=['FlowMean','SatMean'])
ObservationPoint_pivot.sample(9)

FlowMean                                                        \
Time           00:00 00:15 00:30 00:45 01:00 01:15 01:30 01:45 02:00 02:15   
Date                                                                         
24-Dec-2017     3346  3382  2514  2557  2676  2596  2109  2174  1889  1710   
30-Dec-2017     1661  1715  1809  1963  1601  1960  1681  1485  1662  1225   
13-Jan-2018     4133  4231  3932  4025  3678  3344  3166  3069  3275  2870   
11-Jan-2018     3536  3640  3299  3107  2684  2492  1928  2295  2016  1673   
2-Jan-2018      1816  1449  1613  1475  1364  1203  1039  1172   983   805   
18-Dec-2017     2595  2996  2311  1959  2384  1809  1473  1579  1581  1577   
26-Dec-2017     2526  2602  2149  1785  1432  1352  1318   986  1109   861   
25-Dec-2017     2931  3133  2645  2614  2448  2366  1972  1893  1374  1481   
3-Jan-2018      1999  1628  1709  1651  1561  1386  1293  1133  1259   986   

             ... SatMean                                                   \
Time         ...   21:30  21:45  22:00  22:15  22:30  22:45  23:00  23:15   
Date         ...                                                            
24-Dec-2017  ...   53.75  50.00  61.50  67.00  52.25  49.00  49.00  49.50   
30-Dec-2017  ...   46.00  48.25  58.50  40.50  49.25  47.50  56.00  66.25   
13-Jan-2018  ...   75.25  64.50  70.00  73.25  79.75  76.75  69.00  62.00   
11-Jan-2018  ...   69.00  67.75  79.00  70.25  79.25  77.00  76.25  85.75   
2-Jan-2018   ...   44.00  45.25  45.75  55.25  43.75  34.50  32.75  30.00   
18-Dec-2017  ...   64.00  64.00  71.75  59.00  67.00  64.25  58.00  62.50   
26-Dec-2017  ...   63.00  47.75  55.50  63.00  67.75  53.25  54.25  42.00   
25-Dec-2017  ...   72.25  66.00  60.25  64.75  66.25  64.75  54.50  46.00   
3-Jan-2018   ...   49.75  46.50  54.75  37.25  31.50  26.50  22.50  33.00   

                           
Time         23:30  23:45  
Date                       
24-Dec-2017  46.75  49.00  
30-Dec-2017  49.75  40.25  
13-Jan-2018  67.50  71.00  
11-Jan-2018  79.25  72.00  
2-Jan-2018   32.00  39.75  
18-Dec-2017  62.50  53.75  
26-Dec-2017  32.25  33.50  
25-Dec-2017  52.50  37.75  
3-Jan-2018   39.00  38.50  

[9 rows x 192 columns]

In [59]:
TestFlowMean = ObservationPoint_pivot['FlowMean']
TestFlowMean.sample(9)

Time,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Date,,,,,,,,,,,,,,,,,,,,,
9-Jan-2018,2515,2337,2434,1811,1949,1388,1535,1283,1313,1321,...,3904,3946,3829,4198,4124,3812,3487,3854,3534,3078
5-Jan-2018,1968,2829,2070,1865,1773,1684,1538,1466,1537,1379,...,3779,3146,3926,4375,3684,4256,3430,3621,3445,3475
22-Dec-2017,3668,3920,3491,3828,3092,2860,3277,3298,3045,3019,...,3720,3739,3727,3888,3651,3729,4072,4081,3806,3371
25-Dec-2017,2931,3133,2645,2614,2448,2366,1972,1893,1374,1481,...,4640,4177,3649,4056,4172,3997,3433,2893,3145,2421
27-Dec-2017,2201,1763,2103,1857,1597,1683,1514,1354,1168,1217,...,2799,2751,2904,3145,3171,2971,2518,2427,2827,2034
8-Jan-2018,2123,1920,1959,2007,1746,1378,1363,1285,1064,987,...,3290,3287,3279,3823,3599,3371,2748,3086,2791,2805
31-Dec-2017,2413,2616,2531,2633,2546,2701,4012,2373,1628,2593,...,9,61,35,37,22,21,24,9,21,21
15-Jan-2018,2516,2210,2463,2072,1720,1653,1876,1508,1401,1351,...,4382,3361,4054,3795,4059,3704,3329,3269,3798,3477
3-Jan-2018,1999,1628,1709,1651,1561,1386,1293,1133,1259,986,...,3191,2855,3299,2977,3327,2690,2348,2217,2343,2533


In [67]:
TestFlowMean.columns

Index(['00:00', '00:15', '00:30', '00:45', '01:00', '01:15', '01:30', '01:45',
       '02:00', '02:15', '02:30', '02:45', '03:00', '03:15', '03:30', '03:45',
       '04:00', '04:15', '04:30', '04:45', '05:00', '05:15', '05:30', '05:45',
       '06:00', '06:15', '06:30', '06:45', '07:00', '07:15', '07:30', '07:45',
       '08:00', '08:15', '08:30', '08:45', '09:00', '09:15', '09:30', '09:45',
       '10:00', '10:15', '10:30', '10:45', '11:00', '11:15', '11:30', '11:45',
       '12:00', '12:15', '12:30', '12:45', '13:00', '13:15', '13:30', '13:45',
       '14:00', '14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45',
       '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45',
       '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45',
       '20:00', '20:15', '20:30', '20:45', '21:00', '21:15', '21:30', '21:45',
       '22:00', '22:15', '22:30', '22:45', '23:00', '23:15', '23:30', '23:45'],
      dtype='object', name='Time')

In [30]:
# 将以上操作封装成函数
def pivot_process(file_path):
    # 读取数据
    df = pd.read_csv(file_path)
    # 创建日期和时间列
    df = df.pivot_table(index='Date', columns='Time', values=['FlowMean','SatMean'])

    return df

**pivot all the files in mergedcsv to make it easier to process the time series data**

**将mergedcsv中所有文件进行pivot转化，以便进行时间序列分析**

In [31]:

directory_path = '../Data/CongestionScoot/MergedCSVs/'
filenames = get_filenames_without_extension(directory_path)


In [32]:
"""
new_folder_path = '../Data/CongestionScoot/MergedCSVsPivoted'

for id in filenames:
    file_path = f"{directory_path}/{id}.csv"
    df = pivot_process(file_path)
    df.to_csv(f"{new_folder_path}/{id}.csv")
"""

## 时间序列分析 Time Series Analysis

为了使得plot出的三维图表，我们需要对数据进行处理，我们将数据按照时间进行分组，然后对每个时间点的数据进行处理，使得数据变成一个三维的数据。

In [139]:
# 导入某个csv文件进行测试

test = pd.read_csv("../Data/CongestionScoot/MergedCSVsPivoted/00-115.csv",index_col=0)

In [140]:
test_filtered = test.filter(like='FlowMean')

In [141]:
col_name = ['00:00', '00:15', '00:30', '00:45', '01:00', '01:15', '01:30', '01:45',
       '02:00', '02:15', '02:30', '02:45', '03:00', '03:15', '03:30', '03:45',
       '04:00', '04:15', '04:30', '04:45', '05:00', '05:15', '05:30', '05:45',
       '06:00', '06:15', '06:30', '06:45', '07:00', '07:15', '07:30', '07:45',
       '08:00', '08:15', '08:30', '08:45', '09:00', '09:15', '09:30', '09:45',
       '10:00', '10:15', '10:30', '10:45', '11:00', '11:15', '11:30', '11:45',
       '12:00', '12:15', '12:30', '12:45', '13:00', '13:15', '13:30', '13:45',
       '14:00', '14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45',
       '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45',
       '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45',
       '20:00', '20:15', '20:30', '20:45', '21:00', '21:15', '21:30', '21:45',
       '22:00', '22:15', '22:30', '22:45', '23:00', '23:15', '23:30', '23:45']
test_filtered.columns = col_name

In [142]:
test_filtered.drop(index='Time',inplace=True)
test_filtered.drop(index='Date',inplace=True)

C:\Users\SBH\AppData\Local\Temp\ipykernel_21492\1801619425.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SBH\AppData\Local\Temp\ipykernel_21492\1801619425.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [143]:
# 将索引重置为普通列，并为这个列命名为"Date"
# set the index to a normal column and name this column as "Date"
test_filtered = test_filtered.reset_index().rename(columns={'index': 'Date'})

In [116]:
# convert the 'Date' column to datetime format
# 将 'Date' 列转换为日期时间格式
test_filtered['Date'] = pd.to_datetime(test_filtered['Date'])

# change the date format to "dd/mm/yyyy"
# 将日期格式修改为 "dd/mm/yyyy"
test_filtered['Date'] = test_filtered['Date'].dt.strftime('%d/%m/%Y')


In [144]:
# 将 'Date' 列转换为日期时间格式
# 将 'Date' 列转换为日期时间格式
test_filtered['Date'] = pd.to_datetime(test_filtered['Date'])
test_filtered = test_filtered.sort_values(by='Date')
test_filtered.head(5)

,Date,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
24,2017-12-17,517.5,662.5,1223.0,1037.0,1008.0,1177.0,1121.0,1143.0,1041.0,...,929.0,842.0,756.0,751.0,827.0,740.0,778.0,798.0,697.0,698.5
27,2017-12-18,392.0,779.0,613.0,344.0,655.0,481.0,485.0,493.0,361.0,...,816.0,1002.0,848.0,952.0,933.0,975.0,857.0,855.0,1198.0,448.5
30,2017-12-19,431.0,395.0,414.5,870.0,793.0,669.0,663.0,449.0,461.0,...,862.0,869.0,1002.0,1005.0,978.0,1110.0,1008.0,901.0,838.0,446.5
36,2017-12-20,606.0,572.5,478.0,802.0,807.0,833.0,750.0,795.0,696.0,...,914.0,933.0,967.0,1091.0,947.0,1250.0,1112.0,1168.0,1047.0,351.0
39,2017-12-21,586.0,684.3333333333334,521.5,928.0,903.0,718.0,971.0,911.0,767.0,...,958.0,1249.0,1047.0,1015.0,916.0,1083.0,1221.0,1070.0,1102.0,1274.0


In [153]:
#convert every column in test_filtered to float type
#将test_filtered中的每一列转换成float类型
for col in test_filtered.columns[2:]:
    test_filtered[col] = test_filtered[col].astype(float)

In [166]:
# define a function to process the data, merge and split the data into two files named "SatMean" and "FlowMean"
# 定义一个操作，将之前MergedCSVsPivoted文件夹中的所有文件进行处理,将SatMean和FlowMean分别提取出来，并且将Date列转换成日期格式

def Split_Sat_Flow(file_path, new_folder_path1, new_folder_path2):
    for id in filenames:
        # 导入某个csv文件进行测试
        tempfile = pd.read_csv(f"{file_path}/{id}.csv",index_col=0)
        tempfile_Flow = tempfile.filter(like='FlowMean')
        tempfile_Sat = tempfile.filter(like='SatMean')
        col_name = ['00:00', '00:15', '00:30', '00:45', '01:00', '01:15', '01:30', '01:45',
       '02:00', '02:15', '02:30', '02:45', '03:00', '03:15', '03:30', '03:45',
       '04:00', '04:15', '04:30', '04:45', '05:00', '05:15', '05:30', '05:45',
       '06:00', '06:15', '06:30', '06:45', '07:00', '07:15', '07:30', '07:45',
       '08:00', '08:15', '08:30', '08:45', '09:00', '09:15', '09:30', '09:45',
       '10:00', '10:15', '10:30', '10:45', '11:00', '11:15', '11:30', '11:45',
       '12:00', '12:15', '12:30', '12:45', '13:00', '13:15', '13:30', '13:45',
       '14:00', '14:15', '14:30', '14:45', '15:00', '15:15', '15:30', '15:45',
       '16:00', '16:15', '16:30', '16:45', '17:00', '17:15', '17:30', '17:45',
       '18:00', '18:15', '18:30', '18:45', '19:00', '19:15', '19:30', '19:45',
       '20:00', '20:15', '20:30', '20:45', '21:00', '21:15', '21:30', '21:45',
       '22:00', '22:15', '22:30', '22:45', '23:00', '23:15', '23:30', '23:45']
        tempfile_Sat.columns = col_name
        tempfile_Flow.columns = col_name
        tempfile_Sat.drop(index='Time',inplace=True)
        tempfile_Sat.drop(index='Date',inplace=True)
        tempfile_Flow.drop(index='Time',inplace=True)
        tempfile_Flow.drop(index='Date',inplace=True)
        # 将索引重置为普通列，并为这个列命名为"Date"
        tempfile_Sat = tempfile_Sat.reset_index().rename(columns={'index': 'Date'})
        tempfile_Flow = tempfile_Flow.reset_index().rename(columns={'index': 'Date'})
        # 将 'Date' 列转换为日期时间格式
        tempfile_Sat['Date'] = pd.to_datetime(tempfile_Sat['Date'])
        tempfile_Sat = tempfile_Sat.sort_values(by='Date',ascending=True)
        tempfile_Flow['Date'] = pd.to_datetime(tempfile_Flow['Date'])
        tempfile_Flow = tempfile_Flow.sort_values(by='Date',ascending=True)
        #将test_filtered中的每一列转换成float类型
        for col in tempfile_Sat.columns[2:]:
            tempfile_Sat[col] = tempfile_Sat[col].astype(float)
        for col in tempfile_Flow.columns[2:]:
            tempfile_Flow[col] = tempfile_Flow[col].astype(float)

        tempfile_Sat.to_csv(f"{new_folder_path1}/{id}.csv")
        tempfile_Flow.to_csv(f"{new_folder_path2}/{id}.csv")
    print("Splitting completed.")

In [ ]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')

Split_Sat_Flow('../Data/CongestionScoot/MergedCSVsPivoted', '../Data/CongestionScoot/CleanedSatMean', '../Data/CongestionScoot/CleanedFlowMean')

In [145]:
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# 三维画图


In [96]:

# 创建3D图表
fig = go.Figure(data=[go.Surface(z=test_filtered.values[:, 1:], x=test_filtered['Date'], y=test_filtered.columns[1:])])

fig.update_layout(title='日期时间测量值',
                  autosize=False,
                    width=1000, 
                    height=600,
                  margin=dict(l=50, r=50, b=20, t=30))

fig.show()

In [161]:

# initialize the figure as a 3D scatter/line graph
fig = go.Figure()

# iterate every row in the dataframe
for index, row in test_filtered.iterrows():
    # add a line to the graph for each row
    fig.add_trace(go.Scatter3d(
        x=[row['Date']] * len(test_filtered.columns[1:]), # X axis data, use the 'Date' column
        y=test_filtered.columns[1:], # Y axis data, use the column names (excluding 'Date')
        z=row[1:].values, # Z axis data, use the values of the row (excluding the 'Date' column)
        mode='lines', #  use lines to connect the points
        name=f"Row {index}" # the name of the line
    ))

# set the layout
fig.update_layout(
    title='Time Series Data of 00-115', 
    autosize=False, 
    width=1000,
    height=800,
    margin=dict(l=50, r=50, b=30, t=90),
    scene=dict(
        zaxis=dict( 
            range=[0, 2000] 
        )
    ),
    showlegend=False

)


fig.show()
